In [ ]:
import numpy as np
import pandas as pd
import pylab
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import scipy.special

from datetime import datetime
from ipywidgets import interactive, widgets
from matplotlib.widgets import Slider, RangeSlider
from numpy.polynomial.chebyshev import chebfit
from numpy.polynomial import Chebyshev


In [ ]:
df = pd.read_csv("/content/Прошлые данные - PLZL.csv")

In [ ]:
df['Цена'] = df['Цена'].str.replace('.', '').str.replace(',', '.').astype(float)
data = df['Цена']
splitted = df['Дата'].str.split('.', expand=True)
df['день'] = splitted[0].astype('int')
df['месяц'] = splitted[1].astype('int')
df['год'] = splitted[2].astype('int')
t = np.arange(1, len(data)+1, 1)
def ti(n):
  return np.arange(n + 1, len(data)+1, 1)[::-1]
df

,Дата,Цена,Откр.,Макс.,Мин.,Объём,Изм. %,день,месяц,год
0,31.03.2024,13363.5,"12.155,0","13.454,0","12.116,0","2,12M","11,50%",31,3,2024
1,24.03.2024,11985.0,"11.424,5","12.047,5","11.280,5","813,62K","5,08%",24,3,2024
2,17.03.2024,11406.0,"11.786,5","11.928,5","11.255,5","682,10K","-3,01%",17,3,2024
3,10.03.2024,11760.0,"11.759,5","11.934,5","11.578,5","98,75K","0,49%",10,3,2024
4,03.03.2024,11702.5,"11.015,0","11.884,5","11.015,0","943,66K","6,34%",3,3,2024
...,...,...,...,...,...,...,...,...,...,...
200,10.05.2020,12306.0,"11.926,0","12.490,0","11.682,5","678,00K","2,78%",10,5,2020
201,03.05.2020,11973.0,"12.100,0","12.350,0","11.333,0","647,67K","-1,13%",3,5,2020
202,26.04.2020,12110.0,"12.100,0","12.347,5","11.750,0","792,72K","0,12%",26,4,2020
203,19.04.2020,12095.0,"11.630,0","12.500,0","11.203,5","1,50M","3,51%",19,4,2020


In [ ]:
df['Дата'] = pd.to_datetime(df['Дата'], format='%d.%m.%Y')

In [ ]:
# Получение разницы между историческими и полученными данными
def o(n, k, data):
    i, a = 0, 0
    m1 = []
    while i < len(data) - n - 1:
        f = data[i+n+1]
        m1.append(f)
        i += 1
    m2 = k
    m = []
    i = 0
    while i < len(m1) - 1:
        a = abs(round(m1[i] - m2[i], 2))
        m.append(a)
        i += 1
    return m

# Максимальное отклонение
def mo(n, k, data):
    m = o(n, k, data)
    return max(m)


In [ ]:
# Функция для вычисления SMA (Простое скользящее среднее)
def calculate_sma(data, n):
    sma = data.rolling(window=n).mean()
    return sma[::-1]

In [ ]:
def calculate_tema(data, n):
    ema1 = data.ewm(span=n, min_periods=n).mean()
    ema2 = ema1.ewm(span=n, min_periods=n).mean()
    ema3 = ema2.ewm(span=n, min_periods=n).mean()
    tema = 3 * ema1 - 3 * ema2 + ema3
    return tema

In [ ]:
def calculate_lagrange(data, n):
    x = n + 2
    m = []
    while x < len(data) + 2:
        q, w, z = 0.0, 0.0, 0.0
        array1 = []
        array2 = []
        i = x - n - 2
        while i < x - n + 2:
            q = t[i]
            array1.append(q)
            w = data[i]
            array2.append(w)
            i += 1
        a = np.array(array1)
        b = np.array(array2)
        for j in range(len(b)):
            p1 = 1
            p2 = 1
            for i in range(len(a)):
                if i == j:
                    p1 = p1 * 1
                    p2 = p2 * 1
                else:
                    p1 = p1 * (x - a[i])
                    p2 = p2 * (a[j] - a[i])
            z = z + b[j] * (p1 / p2)
        x += 1
        m.append(z)
    return m

In [ ]:
def chebyshev_nodes(n):
    return np.cos((2*np.arange(1, n+1) - 1) * np.pi / (2*n))

def chebyshev_polynomial(x, k):
    return scipy.special.eval_chebyt(k, x)

def chebyshev_coefficients(f, n):
    nodes = chebyshev_nodes(n)
    coeffs = np.zeros(n)
    for k in range(n):
        coeffs[k] = (2/n) * np.sum(f(nodes) * chebyshev_polynomial(nodes, k))
    return coeffs

def chebyshev_approximation(x, coeffs):
    result = np.zeros_like(x)
    for k, coeff in enumerate(coeffs):
        result += coeff * chebyshev_polynomial(x, k)
    return result

def calculate_cheb(data, degree):
    x = np.arange(1, degree + 2)
    x_normalized = np.interp(x, (x.min(), x.max()), (-1, 1))
    x_next = degree + 2
    x_next_normalized = 2 * (x_next - x.min()) / ( x.max() - x.min()) - 1
    result = []
    for i in range(len(data) - degree):
        y = data[i:i+degree+1]
        data_normalized = np.interp(y, (y.min(), y.max()), (-1, 1))
        coefficients = chebyshev_coefficients(lambda x: np.interp(x, np.arange(len(y)), data_normalized), degree)
        approximation = chebyshev_approximation(x_next_normalized, coefficients)
        forecast_next = np.interp(approximation, (-1, 1), (y.min(), y.max()))
        result.append(forecast_next)
    return result

In [ ]:
def find_entry_exit_points(price_data, indicator_data):
    signals = []
    count = 0
    is_buy = True

    for i in range(1, len(price_data)-1):
        # Пересечение индикатора с ценой снизу вверх (точка входа)
        if is_buy:
            if indicator_data[i] > price_data[i] and indicator_data[i-1] <= price_data[i-1]:
                signals.append(('buy', i - 1))
                count += 1
                is_buy = False
        # Пересечение индикатора с ценой сверху вниз (точка выхода)
        else:
             if indicator_data[i] < price_data[i] and indicator_data[i-1] >= price_data[i-1]:
                signals.append(('sell', i - 1))
                count += 1
                is_buy = True
    if signals and signals[0][0] == 'buy':
            del signals[0]
            count -= 1
    if signals and signals[-1][0] == 'sell':
            del signals[-1]
            count -= 1
    return signals, count

In [ ]:
def strategy(signals, df):
    sum, first_buy_sum = 0 , 0
    for signal_type, index in signals:
            first_signal_index = abs(signals[-1][1])
            first_buy_sum = df['Цена'].iloc[first_signal_index]
            if signal_type == 'buy':
                sum -= df['Цена'].iloc[index]
            elif signal_type == 'sell':
                sum += df['Цена'].iloc[index]
    coefficient = round((sum / first_buy_sum) * 100, 3)
    return coefficient

In [ ]:
def find_max_min_coefficient(df, k):
    max_coefficient = -np.inf
    min_coefficient = np.inf
    max_n = None
    min_n = None
    max_count = None
    min_count = None

    for n in range(3, 51):
        if k == 1:
            sma = calculate_sma(df['Цена'][::-1], n)
            signals_data = find_entry_exit_points(df['Цена'], sma)
        if k == 2:
            tema = calculate_tema(df['Цена'].iloc[::-1], n)
            signals_data = find_entry_exit_points(df['Цена'], tema)
        if k == 3:
            lagrange = calculate_lagrange(df['Цена'], n)
            signals_data = find_entry_exit_points(df['Цена'].iloc[:-n], lagrange)
        if k == 4:
            cheb = calculate_cheb(df['Цена'], n)
            signals_data = find_entry_exit_points(df['Цена'].iloc[:-n], cheb)

        signals = signals_data[0]
        coefficient = strategy(signals, df)
        count = signals_data[1]
        if coefficient > max_coefficient:
            max_coefficient = coefficient
            max_n = n
            max_count = count
        if coefficient < min_coefficient:
            min_coefficient = coefficient
            min_n = n
            min_count = count
    return max_coefficient, max_n, max_count, min_coefficient, min_n, min_count


In [ ]:
def plot_graph(start_date, end_date, n, indicator, line_color, grid, strateg):
    df['Дата'] = pd.to_datetime(df['Дата'], format='%d.%m.%Y')
    fig, ax = plt.subplots(figsize=(16, 9))
    ax.plot(df['Дата'], df['Цена'], label='Полюс (PLZL)', color='blue')
    df_sorted = df.sort_values(by='Дата', ascending=True)
    buy_count = 0
    sell_count = 0

    if indicator == 'SMA':
        sma = calculate_sma(df['Цена'][::-1], n)
        ax.plot(df['Дата'], sma, label=f'SMA ({n})', color=line_color)
        differences = np.array(df['Цена'][:-n] - sma[:-n])
        if strateg:
            signals_data = find_entry_exit_points(df['Цена'], sma)
            signals = signals_data[0]
            count = signals_data[1]
            for signal_type, index in signals:
                color = 'green' if signal_type == 'buy' else 'orange'
                if signal_type == 'buy' and buy_count < 1:
                    ax.axvline(df['Дата'].iloc[index], color=color, linestyle='-', label=signal_type)
                    buy_count += 1
                elif signal_type == 'sell' and sell_count < 1:
                    ax.axvline(df['Дата'].iloc[index], color=color, linestyle='-', label=signal_type)
                    sell_count += 1
                else:
                    ax.axvline(df['Дата'].iloc[index], color=color, linestyle='-')
            max_coefficient, max_n, max_count, min_coefficient, min_n, min_count = find_max_min_coefficient(df, 1)
            ax.text(0.02, 1.15, f'Max доходность, %: {max_coefficient}% for n={max_n} with {max_count} operations', transform=ax.transAxes, fontsize=10)
            ax.text(0.02, 1.10, f'Min доходность, %: {min_coefficient}% for n={min_n} with {min_count} operations', transform=ax.transAxes, fontsize=10)
            coefficient = strategy(signals, df)
            ax.text(0.02, 1.25, f'Количество операций: {count}', transform=ax.transAxes)
            ax.text(0.02, 1.20, f'Процентный доход: {coefficient}%', transform=ax.transAxes)
        ax.text(0.02, -0.35, f'Максимальное отклонение: {mo(n, sma, data)}', transform=ax.transAxes)
        ax.text(0.02, -0.40, f'Cреднее отклонение: {round(np.mean(np.abs(differences)), 3)}', transform=ax.transAxes)

    elif indicator == 'TEMA':
        tema = calculate_tema(df['Цена'].iloc[::-1], n)
        ax.plot(df['Дата'], tema[::-1], label=f'TEMA ({n})', color=line_color)
        differences = np.array(df['Цена'][:+3-n*3] - tema[3*n-3:][::-1])
        if strateg:
            signals_data = find_entry_exit_points(df['Цена'], tema) # !!!
            signals = signals_data[0]
            count = signals_data[1]
            for signal_type, index in signals:
                color = 'green' if signal_type == 'buy' else 'orange'
                if signal_type == 'buy' and buy_count < 1:
                    ax.axvline(df['Дата'].iloc[index], color=color, linestyle='-', label=signal_type)
                    buy_count += 1
                elif signal_type == 'sell' and sell_count < 1:
                    ax.axvline(df['Дата'].iloc[index], color=color, linestyle='-', label=signal_type)
                    sell_count += 1
                else:
                    ax.axvline(df['Дата'].iloc[index], color=color, linestyle='-')
            max_coefficient, max_n, max_count, min_coefficient, min_n, min_count = find_max_min_coefficient(df, 2)
            ax.text(0.02, 1.15, f'Max доходность, %: {max_coefficient}% for n={max_n} with {max_count} operations', transform=ax.transAxes, fontsize=10)
            ax.text(0.02, 1.10, f'Min доходность, %: {min_coefficient}% for n={min_n} with {min_count} operations', transform=ax.transAxes, fontsize=10)
            coefficient = strategy(signals, df)
            ax.text(0.02, 1.25, f'Количество операций: {count}', transform=ax.transAxes)
            ax.text(0.02, 1.20, f'Процентный доход: {coefficient}%', transform=ax.transAxes)
        ax.text(0.02, -0.35, f'Максимальное отклонение: {round(np.max(np.abs(differences)), 2)}', transform=ax.transAxes)
        ax.text(0.02, -0.40, f'Cреднее отклонение: {round(np.mean(np.abs(differences)), 2)}', transform=ax.transAxes)

    elif indicator == 'Lagrange':
        lagrange = calculate_lagrange(df['Цена'], n)
        ax.plot(df['Дата'].iloc[:-n], lagrange, label=f'Lagrange ({n})', color=line_color)
        differences = np.array(df['Цена'][n:] - lagrange)
        if strateg:
            signals_data = find_entry_exit_points(df['Цена'].iloc[:-n], lagrange)
            signals = signals_data[0]
            count = signals_data[1]
            for signal_type, index in signals:
                color = 'green' if signal_type == 'buy' else 'orange'
                if signal_type == 'buy' and buy_count < 1:
                    ax.axvline(df['Дата'].iloc[index], color=color, linestyle='-', label=signal_type)
                    buy_count += 1
                elif signal_type == 'sell' and sell_count < 1:
                    ax.axvline(df['Дата'].iloc[index], color=color, linestyle='-', label=signal_type)
                    sell_count += 1
                else:
                    ax.axvline(df['Дата'].iloc[index], color=color, linestyle='-')
            coefficient = strategy(signals, df)
            ax.text(0.02, 1.15, f'Количество операций: {count}', transform=ax.transAxes)
            ax.text(0.02, 1.10, f'Процентный доход: {coefficient}%', transform=ax.transAxes)
            max_coefficient, max_n, max_count, min_coefficient, min_n, min_count = find_max_min_coefficient(df, 3)
            ax.text(0.02, 1.25, f'Max доходность, %: {max_coefficient}% for n={max_n} with {max_count} operations', transform=ax.transAxes, fontsize=10)
            ax.text(0.02, 1.20, f'Min доходность, %: {min_coefficient}% for n={min_n} with {min_count} operations', transform=ax.transAxes, fontsize=10)
        ax.text(0.02, -0.35, f'Максимальное отклонение: {round(np.max(np.abs(differences)), 3)}', transform=ax.transAxes)
        ax.text(0.02, -0.40, f'Cреднее отклонение: {np.mean(np.abs(differences))}', transform=ax.transAxes)

    elif indicator == 'Chebyshev':
        cheb = calculate_cheb(df['Цена'], n)
        ax.plot(df['Дата'][:-n], cheb, label=f'Chebyshev ({n})', color=line_color)
        differences = np.array(df['Цена'][n:] - cheb)
        if strateg:
            signals_data = find_entry_exit_points(df['Цена'].iloc[:-n], cheb)
            signals = signals_data[0]
            count = signals_data[1]
            for signal_type, index in signals:
                color = 'green' if signal_type == 'buy' else 'orange'
                if signal_type == 'buy' and buy_count < 1:
                    ax.axvline(df['Дата'].iloc[index], color=color, linestyle='-', label=signal_type)
                    buy_count += 1
                elif signal_type == 'sell' and sell_count < 1:
                    ax.axvline(df['Дата'].iloc[index], color=color, linestyle='-', label=signal_type)
                    sell_count += 1
                else:
                    ax.axvline(df['Дата'].iloc[index], color=color, linestyle='-')
            coefficient = strategy(signals, df)
            ax.text(0.02, 1.15, f'Количество операций: {count}', transform=ax.transAxes)
            ax.text(0.02, 1.10, f'Процентный доход: {coefficient}%', transform=ax.transAxes)
            max_coefficient, max_n, max_count, min_coefficient, min_n, min_count = find_max_min_coefficient(df, 4)
            ax.text(0.02, 1.25, f'Max доходность, %: {max_coefficient}% for n={max_n} with {max_count} operations', transform=ax.transAxes, fontsize=10)
            ax.text(0.02, 1.20, f'Min доходность, %: {min_coefficient}% for n={min_n} with {min_count} operations', transform=ax.transAxes, fontsize=10)
        ax.text(0.02, -0.35, f'Максимальное отклонение: {round(np.max(np.abs(differences)), 3)}', transform=ax.transAxes)
        ax.text(0.02, -0.40, f'Cреднее отклонение: {np.mean(np.abs(differences))}', transform=ax.transAxes)

    plt.xlim(start_date, end_date)
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%m.%Y'))
    ax.legend()
    ax.set_ylabel('руб.')
    if grid:
        ax.grid(True)
    else:
        ax.grid(False)
    ax.tick_params(axis='x', rotation=45)
    plt.show()


In [ ]:
start_date_widget = widgets.DatePicker(description='Начало')
end_date_widget = widgets.DatePicker(description='Конец')
n_widget = widgets.IntSlider(min=3, max=50, step=1, value=None, description='N')
indicator_widget = widgets.Dropdown(options=['SMA', 'TEMA', 'Lagrange', 'Chebyshev'], value=None, description='Индикатор')
line_color_widget = widgets.ColorPicker(value='red', description='Цвет линии индикатора')
grid_widget = widgets.Checkbox(value=False, description='Отобразить сетку')
how_strategy = widgets.Checkbox(value=False, description='Отобразить стратегию')

interactive(plot_graph, start_date=start_date_widget, end_date=end_date_widget,
                               n=n_widget, indicator=indicator_widget, line_color=line_color_widget, grid=grid_widget, strateg=how_strategy)


interactive(children=(DatePicker(value=None, description='Начало'), DatePicker(value=None, description='Конец'…